# Explore here

In [5]:
# Your code 
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import missingno as msno
from sklearn.impute import KNNImputer
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
from statsmodels.stats.outliers_influence import variance_inflation_factor
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from lazypredict.Supervised import LazyRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import *
from sklearn.ensemble import BaggingRegressor
from sklearn.preprocessing import FunctionTransformer
import joblib


In [6]:
df = pd.read_csv('https://raw.githubusercontent.com/4GeeksAcademy/logistic-regression-project-tutorial/main/bank-marketing-campaign-data.csv', sep=';')
print(df.shape)
df.head().T

(41188, 21)


,0,1,2,3,4
age,56,57,37,40,56
job,housemaid,services,services,admin.,services
marital,married,married,married,married,married
education,basic.4y,high.school,high.school,basic.6y,high.school
default,no,unknown,no,no,no
housing,no,no,yes,no,no
loan,no,no,no,no,yes
contact,telephone,telephone,telephone,telephone,telephone
month,may,may,may,may,may
day_of_week,mon,mon,mon,mon,mon


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp.var.rate    41188 non-null  float64
 16  cons.price.idx  41188 non-null  float64
 17  cons.conf.idx   41188 non-null 

In [8]:
n_unicos = df.nunique()
n_unicos

age                 78
job                 12
marital              4
education            8
default              3
housing              3
loan                 3
contact              2
month               10
day_of_week          5
duration          1544
campaign            42
pdays               27
previous             8
poutcome             3
emp.var.rate        10
cons.price.idx      26
cons.conf.idx       26
euribor3m          316
nr.employed         11
y                    2
dtype: int64

In [9]:
duplicados = df.duplicated().sum()


In [10]:
df = df.drop_duplicates()
print(f"Nuevo número de filas: {len(df)}")

Nuevo número de filas: 41176


In [11]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
age,41176.00,40.02,10.42,17.00,32.00,38.00,47.00,98.00
duration,41176.00,258.32,259.31,0.00,102.00,180.00,319.00,4918.00
campaign,41176.00,2.57,2.77,1.00,1.00,2.00,3.00,56.00
pdays,41176.00,962.46,186.94,0.00,999.00,999.00,999.00,999.00
previous,41176.00,0.17,0.49,0.00,0.00,0.00,0.00,7.00
emp.var.rate,41176.00,0.08,1.57,-3.40,-1.80,1.10,1.40,1.40
cons.price.idx,41176.00,93.58,0.58,92.20,93.08,93.75,93.99,94.77
cons.conf.idx,41176.00,-40.50,4.63,-50.80,-42.70,-41.80,-36.40,-26.90
euribor3m,41176.00,3.62,1.73,0.63,1.34,4.86,4.96,5.04
nr.employed,41176.00,5167.03,72.25,4963.60,5099.10,5191.00,5228.10,5228.10
